# Loading Libraries

In [1248]:
import numpy as np
import json
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [637]:
from IPython.display import HTML
from ipywidgets.widgets import interact, IntSlider, FloatSlider, Layout, ToggleButton

from scipy.integrate import odeint
import numpy as np
import matplotlib.pyplot as plt

import plotly.io as pio

import cufflinks as cf
import plotly.offline
import plotly.graph_objs as go
import plotly.plotly as py

cf.go_offline()
cf.set_config_file(offline=False, world_readable=True)

import os
import sys

import requests
from lmfit import minimize, Parameters, Parameter, report_fit
pio.renderers.default = "notebook"
%matplotlib inline
plt.style.use('ggplot')

style = {'description_width': '100px'}
slider_layout = Layout(width='99%')

pd.options.display.float_format = '{:.2f}'.format

In [638]:
pd.options.display.float_format = '{:.2f}'.format

# Load Files

In [857]:
df1 = pd.read_csv("../covid-19/data/countries-aggregated.csv", sep=',')

In [858]:
df2 = pd.read_csv("../covid-19/data/key-countries-pivoted.csv", sep=',')

In [859]:
df3 = pd.read_csv("../covid-19/data/time-series-19-covid-combined.csv", sep=',')

In [860]:
df4 = pd.read_csv("../covid-19/data/worldwide-aggregated.csv", sep=',')

In [861]:
df5 = pd.read_csv("../covid-19/data/reference.csv", sep=',')

### File Preprocessing

In [862]:
Country_Aggregate = df1[df1['Country']=="China"]

In [863]:
time_series = df3[df3['Country/Region']=="China"]

In [864]:
Worl_Aggregate = df5[df5['Country_Region']=="China"]

In [865]:
Worl_Aggregate = Worl_Aggregate.reset_index()

In [866]:
Country_Aggregate = Country_Aggregate.reset_index()

In [867]:
time_series = time_series.reset_index()

In [868]:
df4 = df4.reset_index()

## BASIC Compartments

In [870]:
result = Worl_Aggregate.sort_values(by='Population', ascending=False).max()

In [871]:
Country_Aggregate["Deaths"] = Country_Aggregate.Deaths.diff().abs()
Country_Aggregate["Confirmed"] = Country_Aggregate.Confirmed.diff().abs()
Country_Aggregate["Recovered"] = Country_Aggregate.Recovered.diff().abs()

Country_Aggregate["Confirmed"] = Country_Aggregate["Confirmed"].fillna(100)
Country_Aggregate["Recovered"] = Country_Aggregate["Recovered"].fillna(1)
Country_Aggregate["Deaths"] = Country_Aggregate["Deaths"].fillna(1)

In [872]:
Country_Aggregate['Susceptible'] = 0
Country_Aggregate['Exposed'] = 0
Country_Aggregate['Infected'] = 0
Country_Aggregate['Infectedundetected'] = 0

In [873]:
a=1404676330.00
for i,v in enumerate(Country_Aggregate['Confirmed']):
    a = a-(Country_Aggregate['Confirmed'][i]+Country_Aggregate['Deaths'][i])
    Country_Aggregate['Susceptible'][i]=a

In [874]:
# Country_Aggregate['Susceptible'] = Country_Aggregate['Susceptible'].diff().abs()
Country_Aggregate['Exposed'] = Country_Aggregate['Susceptible'].diff().abs()
Country_Aggregate['Infected'] = Country_Aggregate['Susceptible'].diff().abs()
Country_Aggregate['Infectedundetected'] = (Country_Aggregate['Infected'] - Country_Aggregate['Confirmed']).abs()

In [875]:
Country_Aggregate["Susceptible"] = Country_Aggregate["Susceptible"].fillna(100)
Country_Aggregate["Exposed"] = Country_Aggregate["Exposed"].fillna(Country_Aggregate["Exposed"].mean())
Country_Aggregate["Infected"] = Country_Aggregate["Infected"].fillna(Country_Aggregate["Infected"].mean())
Country_Aggregate["Infectedundetected"] = Country_Aggregate["Infectedundetected"].fillna(Country_Aggregate["Infectedundetected"].mean())

In [876]:
Country_Aggregate.Date = pd.to_datetime(Country_Aggregate.Date)

# COVID-19 Susceptible Monthly

In [877]:
Country_Aggregate.groupby(Country_Aggregate['Date'].dt.month)[["Susceptible"]].sum().iplot(kind='scatter',title = 'COVID-19 Susceptible Monthly')

# Confirmed, Recovered, Deaths, Exposed, Infected, Infectedundetected By Month

In [878]:
Country_Aggregate.groupby(Country_Aggregate['Date'].dt.month)[["Confirmed","Recovered","Deaths","Exposed","Infected","Infectedundetected"]].sum().iplot(kind='bar',title = 'COVID-19 By Month')

# Confirmed, Recovered, Deaths, Exposed, Infected, Infectedundetected By Month

In [879]:
Country_Aggregate[["Confirmed","Recovered","Deaths","Exposed","Infected","Infectedundetected"]].iplot(kind='scatter',title = 'COVID-19 By days')

### Compartments

In [922]:
# S
Susceptible_compartment = float(result.Population)
# E
Exposed_compartment = float(Country_Aggregate.Exposed.max())
# I
Infected_compartment = float(Country_Aggregate.Infected.max())
# Iu
Infectedundetected_compartment = float(Country_Aggregate.Infectedundetected.max()) 
# H
Hospitalized_compartment = float(Country_Aggregate.Confirmed.max())
# R
Recovered_compartment = float(Country_Aggregate.Recovered.max())
# D
Death_compartment = float(Country_Aggregate.Deaths.max())

print('Susceptible:',Susceptible_compartment)
print('Exposed:',Exposed_compartment)
print('Infected:',Infected_compartment)
print('Infectedundetected:',Infectedundetected_compartment)
print('Hospitalized:',Hospitalized_compartment)
print('Recovered:',Recovered_compartment)
print('Death:',Death_compartment)

Susceptible: 1404676330.0
Exposed: 15388.0
Infected: 15388.0
Infectedundetected: 1290.0
Hospitalized: 15136.0
Recovered: 3995.0
Death: 1290.0


# Parameters Used In Notebook

### mE  , mI , mIu , mHR , mHD Control Measures

### μm Mortality Rate

### θ Total Infected People detected

### γ Transition rate from one compartment to another

### β Disease contatct rate in compartments

### τ1 Arrives to territory i from other territories per day | τ2 leaves territory i per day

# CASE 1 : S -> E -> I -> Iu -> Ru

### Parameters

In [951]:
MUe = 1.0
MUi = 1.0
MUiu = 1.0
days = len(Country_Aggregate.Date.unique())

initE = 1000
initI = Country_Aggregate.Confirmed.min()
initIu = 0
initRu = 0
initN = 1
initDays = 1

### Model Function

In [952]:
def SEIR(x, t):
    S, E, I, Iu, Ru, betae, betai, betaiu, N, Tao1, Tao2, Gammae, Gammai, Gammaiu, Theta, MUe, MUi, MUiu, Omega = x
    
    dS = (-(S/N)*((MUe*betae*E)+(MUi*betai*I)+(MUiu*betaiu*Theta*Iu)))
    
    dE = ((S/N)*((MUe*betae*E)+(MUi*betai*I)+(MUiu*betaiu*Theta*Iu))) - Gammae*E + Tao1 - Tao2
    
    dI = (Gammae*E) - (Gammai*I)
    
    dIu = ((Gammai*I)-(Gammai*I*Theta)-(Gammaiu*Iu))
    
    dRu = (Gammaiu*Iu)
    
    return [dS, dE, dI, dIu, dRu, betae, betai, betaiu, N, Tao1, Tao2, Gammae, Gammai, Gammaiu, Theta, MUe, MUi, MUiu, Omega]

### Main Function

In [955]:
def main(initE, initI, initIu, initRu, initN, initMUe, initMUi, initMUiu, days, param_fitting):
    
    MUe = initMUe
    MUi = initMUi
    MUiu = initMUiu
    
    Theta = float(Hospitalized_compartment)
    
    Gammae = MUe
    Gammai = MUi
    Gammaiu = MUiu
    
    Tao1 = 0#float(Country_Aggregate.Exposed.max())
    Tao2 = 0#float(Country_Aggregate.Exposed.max())

    tspan = np.arange(0, days, 1)
    
    betae = 1.0 / MUe 
    betai = 1.0 / MUi
    betaiu = 1.0 / MUiu

    R0 = ((MUi*betai) / (Gammai * (Theta+1)))
    print("R0:" , R0)
    
    betae = R0*Gammae          
    betai = R0*Gammai
    betaiu = R0*Gammaiu
    
    Omega = 0.0527
    
    N = initN

    
    IC = [1000, 1000, 500, Country_Aggregate.Infectedundetected.min(),Country_Aggregate.Recovered.min(),
         betae, betai, betaiu, N, Tao1, Tao2, Gammae, Gammai, Gammaiu, Theta, MUe, MUi, MUiu, Omega]
    t = np.linspace(0, 180, 1000)
    soln = odeint(SEIR, IC, t)
    S, E, I, Iu, Ru = soln[:, 0], soln[:, 1], soln[:, 2]+soln[:, 3], soln[:, 3], soln[:, 4]
    
    S[S < 0] = 0
    E[E < 0] = 0
    I[I < 0] = 0
    Iu[Iu < 0] = 0
    Ru[Ru < 0] = 0
    
    # Create traces
    fig = go.Figure()
    if not param_fitting:
        fig.add_trace(go.Scatter(x=tspan, y = S, mode='lines+markers', name='Susceptible'))
        fig.add_trace(go.Scatter(x=tspan, y = E, mode='lines+markers', name='Exposed'))
    fig.add_trace(go.Scatter(x=tspan, y = I, mode='lines+markers', name='Infected'))
    fig.add_trace(go.Scatter(x=tspan, y = Iu, mode='lines+markers',name='Infected Undetected'))
    fig.add_trace(go.Scatter(x=tspan, y = Ru, mode='lines+markers',name='Recovered Undetected'))

    if param_fitting:
        fig.add_trace(go.Scatter(x=tspan, y=Country_Aggregate.Confirmed, mode='lines+markers',\
                             name='Infections Observed', line = dict(dash='dash')))
        fig.add_trace(go.Scatter(x=tspan, y=Country_Aggregate.Recovered, mode='lines+markers',\
                             name='Hospitalized Recovered', line = dict(dash='dash')))
        fig.add_trace(go.Scatter(x=tspan, y=Country_Aggregate.Deaths, mode='lines+markers',\
                             name='Deaths Observed', line = dict(dash='dash')))
    
    if days <= 30:
        step = 1
    elif days <= 90:
        step = 7
    else:
        step = 30
    
    # Edit the layout
    fig.layout.update(title='Simulation of S E I Iu Ru Model',
                       xaxis_title='Day',
                       yaxis_title='Counts',
                       title_x=0.5,
                      width=900, height=600
                     )
    fig.update_xaxes(tickangle=-90, tickformat = None, tickmode='array', tickvals=np.arange(0, days + 1, step))
    if not os.path.exists("images"):
        os.mkdir("images")
    fig.write_image("images/seird_simulation.png")
    fig.show()

### Slider Widget

In [956]:
interact(main, 
         initE = IntSlider(min = 0, max = Exposed_compartment, step = 1, value = initE, description = 'E', style = style, layout = slider_layout),
         initI = IntSlider(min = 0, max = Infected_compartment, step = 10, value = initI, description = 'I', style = style, layout = slider_layout),
         initIu = IntSlider(min = 0, max = Infectedundetected_compartment, step = 10, value = initIu, description = 'Iu', style = style, layout = slider_layout),
         initRu = IntSlider(min = 0, max = Recovered_compartment, step = 10, value = initIu, description = 'Ru', style = style, layout = slider_layout),
         initN = IntSlider(min = 0, max = result.Population, step = 10, value = initN, description = 'N', style = style, layout = slider_layout),
         initMUe = FloatSlider(min = 0, max = 4, step = 0.01, value = MUe, description = 'Control Measure Exposed', style = style, layout = slider_layout),
         initMUi = FloatSlider(min = 0, max = 4, step = 0.01, value = MUi, description = 'Control Measure Infected', style = style, layout = slider_layout),
         initMUiu = FloatSlider(min = 0, max = 4, step = 0.01, value = MUhr, description = 'Control Measure Hospitalized Recovered', style = style, layout = slider_layout),
         days = IntSlider(min = 0, max = len(Country_Aggregate.Date.unique()), step = 7, value = initDays, description = 'Days', style = style, layout = slider_layout),
         param_fitting = ToggleButton(value = False, description = 'Fitting Mode', disabled = False, button_style = '', \
             tooltip = 'Click to show fewer plots', icon = 'check-circle')
        );

interactive(children=(IntSlider(value=1000, description='E', layout=Layout(width='99%'), max=15388, style=Slid…

# CASE 2 : S -> E -> I -> Hr -> Rd

### Parameters used

In [760]:
MUe = 1.0
MUi = 1.0
MUhr = 1.0
days = len(Country_Aggregate.Date.unique())

initE = 1000
initI = Country_Aggregate.Confirmed.min()
initHr = 0
initRd = 0
initN = 1
initDays = 1

### Model Function

In [761]:
def SEIR(x, t):
    S, E, I, Hr, Rd, betae, betai, betahr, N, Tao1, Tao2, Gammae, Gammai, Gammahr, Theta, MUe, MUi, MUhr, Omega = x
    
    dS = (-(S/N)*((MUe*betae*E)+(MUi*betai*I))) - ((S/N)*(MUhr*betahr*Hr))
    
    dE = ((S/N)*((MUe*betae*E)+(MUi*betai*I))) + ((S/N)*(MUhr*betahr*Hr)) - Gammae*E + Tao1 - Tao2
    
    dI = (Gammae*E) - (Gammai*I)
    
    dHr = (Theta*Gammai*I) - (Omega*Gammai*I) - (Gammahr*Hr)
    
    dRd = (Gammahr*Hr)
    
    return [dS, dE, dI, dHr, dRd, betae, betai, betahr, N, Tao1, Tao2, Gammae, Gammai, Gammahr, Theta, MUe, MUi, MUhr, Omega]

### Main Function

In [959]:
def main(initE, initI, initHr, initRd, initN, initMUe, initMUi, initMUhr, days, param_fitting):
    
    MUe = initMUe
    MUi = initMUi
    MUhr = initMUhr
    
    Theta = float(Hospitalized_compartment)
    
    Gammae = MUe
    Gammai = MUi
    Gammahr = MUhr
    
    Tao1 = 0#float(Country_Aggregate.Exposed.max())
    Tao2 = 0#float(Country_Aggregate.Exposed.max())

    tspan = np.arange(0, days, 1)
    
    betae = 1.0 / MUe 
    betai = 1.0 / MUi
    betahr = 1.0 / MUhr

    R0 = ((MUi*betai)/Gammai)
    print("R0 : ", R0)
    
    betae = R0*Gammae          
    betai = R0*Gammai
    betahr = R0*Gammahr
    
    Omega = 0.0527
    
    N = initN

    
    IC = [1000, 1000, 500, Country_Aggregate.Confirmed.min(),Country_Aggregate.Recovered.min(),
         betae, betai, betahr, N, Tao1, Tao2, Gammae, Gammai, Gammahr, Theta, MUe, MUi, MUhr, Omega]
    t = np.linspace(0, 180, 1000)
    soln = odeint(SEIR, IC, t)
    S, E, I, Hr, Rd = soln[:, 0], soln[:, 1], soln[:, 2], soln[:, 3], soln[:, 4]

    
    # Create traces
    fig = go.Figure()
    if not param_fitting:
        fig.add_trace(go.Scatter(x=tspan, y = S, mode='lines+markers', name='Susceptible'))
        fig.add_trace(go.Scatter(x=tspan, y = E, mode='lines+markers', name='Exposed'))
    fig.add_trace(go.Scatter(x=tspan, y = I, mode='lines+markers', name='Infected'))
    fig.add_trace(go.Scatter(x=tspan, y = Hr, mode='lines+markers',name='Hospitalized Recovered'))
    fig.add_trace(go.Scatter(x=tspan, y = Rd, mode='lines+markers',name='Recovered detected'))

    if param_fitting:
        fig.add_trace(go.Scatter(x=tspan, y=Country_Aggregate.Confirmed, mode='lines+markers',\
                             name='Infections Observed', line = dict(dash='dash')))
        fig.add_trace(go.Scatter(x=tspan, y=Country_Aggregate.Recovered, mode='lines+markers',\
                             name='Hospitalized Recovered', line = dict(dash='dash')))
        fig.add_trace(go.Scatter(x=tspan, y=Country_Aggregate.Deaths, mode='lines+markers',\
                             name='Deaths Observed', line = dict(dash='dash')))
    
    if days <= 30:
        step = 1
    elif days <= 90:
        step = 7
    else:
        step = 30
    
    # Edit the layout
    fig.layout.update(title='Simulation of S E I Hr Rd Model',
                       xaxis_title='Day',
                       yaxis_title='Counts',
                       title_x=0.5,
                      width=900, height=600
                     )
    fig.update_xaxes(tickangle=-90, tickformat = None, tickmode='array', tickvals=np.arange(0, days + 1, step))
    if not os.path.exists("images"):
        os.mkdir("images")
    fig.write_image("images/seird_simulation.png")
    fig.show()

### Slider Widget

In [961]:
interact(main, 
         initE = IntSlider(min = 0, max = Exposed_compartment, step = 1, value = initE, description = 'E', style = style, layout = slider_layout),
         initI = IntSlider(min = 0, max = Infected_compartment, step = 10, value = initI, description = 'I', style = style, layout = slider_layout),
         initHr = IntSlider(min = 0, max = Recovered_compartment, step = 10, value = initIu, description = 'Hr', style = style, layout = slider_layout),
         initRd = IntSlider(min = 0, max = Recovered_compartment, step = 10, value = initIu, description = 'Rd', style = style, layout = slider_layout),
         initN = IntSlider(min = 0, max = result.Population, step = 10, value = initN, description = 'N', style = style, layout = slider_layout),
         initMUe = FloatSlider(min = 0, max = 4, step = 0.01, value = MUe, description = 'Control Measure Exposed', style = style, layout = slider_layout),
         initMUi = FloatSlider(min = 0, max = 4, step = 0.01, value = MUi, description = 'Control Measure Infected', style = style, layout = slider_layout),
         initMUhr = FloatSlider(min = 0, max = 4, step = 0.01, value = MUhr, description = 'Control Measure Hospitalized Recovered', style = style, layout = slider_layout),
         days = IntSlider(min = 0, max = len(Country_Aggregate.Date.unique()), step = 7, value = initDays, description = 'Days', style = style, layout = slider_layout),
         param_fitting = ToggleButton(value = False, description = 'Fitting Mode', disabled = False, button_style = '', \
             tooltip = 'Click to show fewer plots', icon = 'check-circle')
        );

interactive(children=(IntSlider(value=1000, description='E', layout=Layout(width='99%'), max=15388, style=Slid…

# CASE 3 : S -> E -> I -> HD -> D

### Parameters Used

In [1095]:
MUe = 1.0
MUi = 1.0
MUhd = 1.0
days = len(Country_Aggregate.Date.unique())

initE = 1000
initI = Country_Aggregate.Confirmed.min()
initHD = 0
initD = 0
initN = 1
initDays = 1

### Model Function

In [1096]:
def SEIR(x, t):
    S, E, I, HD, D, betae, betai, betahd, N, Tao1, Tao2, Gammae, Gammai, Gammahd, Theta, MUe, MUi, MUhd, Omega = x
    
    dS = (-(S/N)*((MUe*betae*E)+(MUi*betai*I))) - ((S/N)*(MUhd*betahd*HD))
    
    dE = ((S/N)*((MUe*betae*E)+(MUi*betai*I))) + ((S/N)*(MUhd*betahd*HD)) - Gammae*E + Tao1 - Tao2
    
    dI = (Gammae*E) - (Gammai*I)
    
    dHD = (Omega*Gammai*I)-(Gammahd*HD)
    
    dD = (Gammahd*HD)
    
    return [dS, dE, dI, dHD, dD, betae, betai, betahd, N, Tao1, Tao2, Gammae, Gammai, Gammahd, Theta, MUe, MUi, MUhd, Omega]

### Main Function

In [1097]:
def main(initE, initI, initHD, initD, initN, initMUe, initMUi, initMUhd, days, param_fitting):
    
    MUe = initMUe
    MUi = initMUi
    MUhd = initMUhd
    
    Theta = float(Hospitalized_compartment)
    
    Gammae = MUe
    Gammai = MUi
    Gammahd = MUhd
    
    Tao1 = 0#float(Country_Aggregate.Exposed.max())
    Tao2 = 0#float(Country_Aggregate.Exposed.max())

    tspan = np.arange(0, days, 1)
    
    betae = 1.0 / MUe 
    betai = 1.0 / MUi
    betahd = 1.0 / MUhd

    R0 = ((MUe*betae)/Gammae)
    print("R0 :", R0)
    
    betae = R0*Gammae          
    betai = R0*Gammai
    betahd = R0*Gammahd
    
    Omega = 0.0527
    
    N = initN

    
    IC = [1000, 1000, 500, Country_Aggregate.Confirmed.min(),Country_Aggregate.Recovered.min(),
         betae, betai, betahd, N, Tao1, Tao2, Gammae, Gammai, Gammahd, Theta, MUe, MUi, MUhd, Omega]
    t = np.linspace(0, 180, 1000)
    soln = odeint(SEIR, IC, t)
    S, E, I, HD, D = soln[:, 0], soln[:, 1], soln[:, 2], soln[:, 3]+soln[:, 4], soln[:, 4]

    
    # Create traces
    fig = go.Figure()
    if not param_fitting:
        fig.add_trace(go.Scatter(x=tspan, y = S, mode='lines+markers', name='Susceptible'))
        fig.add_trace(go.Scatter(x=tspan, y = E, mode='lines+markers', name='Exposed'))
    fig.add_trace(go.Scatter(x=tspan, y = I, mode='lines+markers', name='Infected'))
    fig.add_trace(go.Scatter(x=tspan, y = HD, mode='lines+markers',name='Hospitalized Death'))
    fig.add_trace(go.Scatter(x=tspan, y = D, mode='lines+markers',name='Death'))

    if param_fitting:
        fig.add_trace(go.Scatter(x=tspan, y=Country_Aggregate.Confirmed, mode='lines+markers',\
                             name='Infections Observed', line = dict(dash='dash')))
        fig.add_trace(go.Scatter(x=tspan, y=Country_Aggregate.Recovered, mode='lines+markers',\
                             name='Hospitalized Recovered', line = dict(dash='dash')))
        fig.add_trace(go.Scatter(x=tspan, y=Country_Aggregate.Deaths, mode='lines+markers',\
                             name='Deaths Observed', line = dict(dash='dash')))
    
    if days <= 30:
        step = 1
    elif days <= 90:
        step = 7
    else:
        step = 30
    
    # Edit the layout
    fig.layout.update(title='Simulation of S E I HD D Model',
                       xaxis_title='Day',
                       yaxis_title='Counts',
                       title_x=0.5,
                      width=900, height=600
                     )
    fig.update_xaxes(tickangle=-90, tickformat = None, tickmode='array', tickvals=np.arange(0, days + 1, step))
    if not os.path.exists("images"):
        os.mkdir("images")
    fig.write_image("images/seird_simulation.png")
    fig.show()

### Slider Widget

In [1098]:
interact(main, 
         initE = IntSlider(min = 0, max = Exposed_compartment, step = 1, value = initE, description = 'E', style = style, layout = slider_layout),
         initI = IntSlider(min = 0, max = Infected_compartment, step = 10, value = initI, description = 'I', style = style, layout = slider_layout),
         initHD = IntSlider(min = 0, max = Hospitalized_compartment, step = 10, value = initIu, description = 'HD', style = style, layout = slider_layout),
         initD = IntSlider(min = 0, max = Death_compartment, step = 10, value = initIu, description = 'D', style = style, layout = slider_layout),
         initN = IntSlider(min = 0, max = result.Population, step = 10, value = initN, description = 'N', style = style, layout = slider_layout),
         initMUe = FloatSlider(min = 0, max = 4, step = 0.01, value = MUe, description = 'Control Measure Exposed', style = style, layout = slider_layout),
         initMUi = FloatSlider(min = 0, max = 4, step = 0.01, value = MUi, description = 'Control Measure Infected', style = style, layout = slider_layout),
         initMUhd = FloatSlider(min = 0, max = 4, step = 0.01, value = MUhr, description = 'Control Measure Hospitalized Death', style = style, layout = slider_layout),
         days = IntSlider(min = 0, max = len(Country_Aggregate.Date.unique()), step = 7, value = initDays, description = 'Days', style = style, layout = slider_layout),
         param_fitting = ToggleButton(value = False, description = 'Fitting Mode', disabled = False, button_style = '', \
             tooltip = 'Click to show fewer plots', icon = 'check-circle')
        );

interactive(children=(IntSlider(value=1000, description='E', layout=Layout(width='99%'), max=15388, style=Slid…

# Final Compartment

# S, E, I, Iu , HR , HD , Rd , Ru or D

In [1350]:
np.set_printoptions(suppress=True,
   formatter={'float_kind':'{:16.3f}'.format}, linewidth=130)

In [1370]:
#Country_Aggregate

### Parameters Used

In [1366]:
MUe = 2.0
MUi = 2.0
MUhd = 2.0
MUiu = 2.0
MUhr = 2.0
days = len(Country_Aggregate.Date.unique())

initE = 100 # Exposed.min()
initI = 1 # Infected.min()
initIu = 1 # Infectedundetected.min()
initHR = 1 # Recovered.min()
initHD = 1 # Deaths.min()
initRd = 1 # Recovered.min()
initRu = 1 # Infectedundetected.min()
initD = 1 # Deaths.min()
initN = result.Population
initDays = 1

### Model Function

In [1367]:
def SEIR(x, t):
    S, E, I, Iu, HR, HD, Rd, Ru, D, betae, betai, betahd, betaiu, betahr, N, Tao1, Tao2, Gammae, Gammai, Gammahd, Gammahr, Gammaiu, Theta, MUe, MUi, MUhd, MUiu, MUhr, Omega = x
   
    dS = (-(S/N)*((MUe*betae*E)+(MUi*betai*I)+(MUiu*betaiu*Theta*Iu))) - ((S/N)*((MUhr*betahr*HR)+(MUhd*betahd*HD))) 
    dS = abs(dS)
    
    dE = ((S/N)*((MUe*betae*E)+(MUi*betai*I)+(MUiu*betaiu*Theta*Iu))) + ((S/N)*(MUhr*betahr*HR)+(MUhd*betahd*HD)) - (Gammae*E) + Tao1 - Tao2
    dE = abs(dE)
    
    dI = (Gammae*E)-(Gammai*I)
    dI = abs(dI)
    
    dIu = ((1-Theta)*Gammai*I) - (Gammaiu*Iu)
    dIu = abs(dIu)
    
    dHR = (Theta*Gammai*I)-(Omega*Gammai*I)-(Gammahr*HR)
    dHR = abs(dHR)
    
    dHD = (Omega*Gammai*I)-(Gammahd*HD)
    dHD = abs(dHD)
    
    dRd = (Gammahr*HR)
    dRd = abs(dRd)
    
    dRu = (Gammaiu*Iu)
    dRu = abs(dRu)
    
    dD = (Gammahd*HD)
    dD = abs(dD)
    
    return [dS, dE, dI, dIu, dHR, dHD, dRd, dRu, dD, betae, betai, betahd, betaiu, betahr, N, Tao1, Tao2, Gammae, Gammai, Gammahd, Gammahr, Gammaiu, Theta, MUe, MUi, MUhd, MUiu, MUhr, Omega]

### Main Function

In [1368]:
def main(initE, initI, initIu, initHR, initHD, initRd, initRu, initD, initN, initMUe, initMUi, initMUhd, initMUiu, initMUhr, days, param_fitting):
    MUe = initMUe
    MUi = initMUi
    MUhd = initMUhd
    MUiu = initMUiu 
    MUhr = initMUhr
    
    Theta = 0.65
    
    Gammae = 0.1818
    Gammai = 0.1493
    Gammahd = MUhd
    Gammahr = 0.0752
    Gammaiu = 0.0752
    
    Tao1 = 0#float(Country_Aggregate.Exposed.max())
    Tao2 = 0#float(Country_Aggregate.Exposed.max())

    tspan = np.arange(0, days, 1)
    
    betae = 0.1079 / MUe 
    betai = 0.2834 / MUi
    betahd = 0.0924 / MUhd
    betaiu = 0.2650 / MUiu
    betahr = 0.0924 / MUhr
    
    Omega = 0.0161

    
    R0 = ((((((betaiu*(1-Theta)*Gammahr)+((betahr*Gammaiu)*(Theta-Omega)))*Gammai + (betai*Gammahr*Gammaiu))*Gammae + (betae*Gammai*Gammahr*Gammaiu))*Gammahd)+(betahd*Omega*Gammae*Gammai*Gammahr*Gammaiu))/(Gammae*Gammai*Gammahd*Gammahd*Gammahr*Gammaiu)
    print('R0:', R0)
    betae = R0 * Gammae          
    betai = R0 * Gammai
    betahd = R0 * Gammahd
    betaiu = R0 * Gammaiu
    betahr = R0 * Gammahr
    
    
    N = initN

    IC = [N, initE, initI,initIu, initHR, initHD,initRd, initRu, initD,
         betae, betai, betahd, betaiu, betahr, N, Tao1, Tao2, Gammae, Gammai, Gammahd, Gammahr, Gammaiu,
         Theta, MUe, MUi, MUhd, MUiu, MUhr, Omega]
    
    t = np.linspace(0, 180, 1000)
    soln = odeint(SEIR, IC, t)
    
    S, E, I, Iu, HR, HD, Rd, Ru, D = soln[:, 0], soln[:, 1], soln[:, 2]+soln[:, 3], soln[:, 3], soln[:, 4], soln[:, 5], soln[:, 6], soln[:, 7], soln[:, 8] 
    S = S**-3
    E = E**-3
    I = I**-3
    Iu = Iu**-3
    HR = HR**-3
    HD = HD**-3
    Rd = Rd**-3
    Ru = Ru**-3
    D = D**-3

    
    fig = go.Figure()
    if not param_fitting:
        np.set_printoptions(formatter={'float_kind':'{:f}'.format})
        fig.add_trace(go.Scatter(x=tspan, y = S, mode='lines+markers', name='Susceptible'))     
        fig.add_trace(go.Scatter(x=tspan, y = E, mode='lines+markers', name='Exposed'))
        fig.add_trace(go.Scatter(x=tspan, y = I, mode='lines+markers', name='Infected'))
        fig.add_trace(go.Scatter(x=tspan, y = Iu, mode='lines+markers',name='Infected Undetected'))
        fig.add_trace(go.Scatter(x=tspan, y = HR, mode='lines+markers',name='Hospitalized Recovered'))
        fig.add_trace(go.Scatter(x=tspan, y = HD, mode='lines+markers',name='Hospitalized Death'))
        fig.add_trace(go.Scatter(x=tspan, y = Rd, mode='lines+markers',name='Recovered Detected'))
        fig.add_trace(go.Scatter(x=tspan, y = Ru, mode='lines+markers',name='Hospitalized Death'))
        fig.add_trace(go.Scatter(x=tspan, y = D, mode='lines+markers',name='Death'))
    
    if param_fitting:
        fig.add_trace(go.Bar(x=tspan, y=Country_Aggregate.Confirmed, mode='lines+markers',\
                             name='Infections Observed', line = dict(dash='dash')))
        fig.add_trace(go.Bar(x=tspan, y=Country_Aggregate.Recovered, mode='lines+markers',\
                             name='Hospitalized Recovered', line = dict(dash='dash')))
        fig.add_trace(go.Bar(x=tspan, y=Country_Aggregate.Deaths, mode='lines+markers',\
                             name='Deaths Observed', line = dict(dash='dash')))

    if days <= 30:
        step = 1
    elif days <= 90:
        step = 7
    else:
        step = 30
    
    # Edit the layout
    fig.layout.update(title='Simulation of S-E-I-IU-Ru-HR-Rd-HD-D Model',
                       xaxis_title='Day',
                       yaxis_title='Counts',  
                       title_x=0.5,
                      width=900, height=600
                     )
    fig.update_xaxes(tickangle=-90, tickformat = None, tickmode='array', tickvals=np.arange(0, days + 1, step))
    if not os.path.exists("images"):
        os.mkdir("images")
    fig.write_image("images/seird_simulation.png")
    fig.show()

### Slider Widget

In [1369]:
interact(main, 
         initE = IntSlider(min = 0, max = Exposed_compartment, step = 10, value = initE, description = 'E', style = style, layout = slider_layout),
         
         initI = IntSlider(min = 0, max = Infected_compartment, step = 10, value = initI, description = 'I', style = style, layout = slider_layout),
         
         initIu = IntSlider(min = 0, max = Infectedundetected_compartment, step = 10, value = initIu, description = 'Iu', style = style, layout = slider_layout), 
         
         initHR = IntSlider(min = 0, max = Recovered_compartment, step = 10, value = initHR, description = 'HR', style = style, layout = slider_layout),
         
         initHD = IntSlider(min = 0, max = Death_compartment, step = 10, value = initHD, description = 'HD', style = style, layout = slider_layout),
         
         initRd = IntSlider(min = 0, max = Recovered_compartment, step = 10, value = initRd, description = 'Rd', style = style, layout = slider_layout), 
         
         initRu = IntSlider(min = 0, max = Recovered_compartment, step = 10, value = initRu, description = 'Ru', style = style, layout = slider_layout),
         
         initD = IntSlider(min = 0, max = Death_compartment, step = 10, value = initD, description = 'D', style = style, layout = slider_layout),
         
         initN = IntSlider(min = 0, max = result.Population, step = 10, value = initN, description = 'N', style = style, layout = slider_layout),
         initMUe = FloatSlider(min = 0, max = 100, step = 1, value = MUe, description = 'Control Measure Exposed', style = style, layout = slider_layout),
         initMUi = FloatSlider(min = 0, max = 100, step = 1, value = MUi, description = 'Control Measure Infected', style = style, layout = slider_layout),
         initMUhd = FloatSlider(min = 0, max = 100, step = 1, value = MUhd, description = 'Control Measure Hospitalized Detected', style = style, layout = slider_layout),
         initMUiu = FloatSlider(min = 0, max = 100, step = 1, value = MUiu, description = 'Control Measure Infected Undetected', style = style, layout = slider_layout),
         initMUhr = FloatSlider(min = 0, max = 100, step = 1, value = MUhr, description = 'Control Measure Hospitalized Recovered', style = style, layout = slider_layout),
         days = IntSlider(min = 0, max = 600, step = 1, value = initDays, description = 'Days', style = style, layout = slider_layout),
         param_fitting = ToggleButton(value = False, description = 'Fitting Mode', disabled = False, button_style = '', \
             tooltip = 'Click to show fewer plots', icon = 'check-circle')
        );

interactive(children=(IntSlider(value=100, description='E', layout=Layout(width='99%'), max=15388, step=10, st…